In [1]:
#Dependencies
import pandas as pd
import requests
import json
from config import api_key


In [2]:
states_abbrev_file_path = "Resources/states_abbrev.csv"

states_abbrev_df = pd.read_csv(states_abbrev_file_path, encoding='ISO-8859-1')
states_abbrev_df

,Airport,City,Abbrev,State,IATA
0,Aberdeen Regional Airport,Aberdeen,SD,South Dakota,ABR
1,Abilene Regional Airport,Abilene,TX,Texas,ABI
2,Abraham Lincoln Capital Airport,Springfield,IL,Illinois,SPI
3,Akron-Canton Regional Airport,Akron / Canton,OH,Ohio,CAK
4,Albany International Airport,Albany,NY,New York,ALB
...,...,...,...,...,...
367,Yampa Valley Airport,Hayden,CO,Colorado,HDN
368,Yeager Airport,Charleston,WV,West Virginia,CRW
369,Yellowstone Regional Airport,Cody,WY,Wyoming,COD
370,Youngstown-Warren Regional Airport,Youngstown / Warren,OH,Ohio,YNG


In [3]:
states_abbrev_df = states_abbrev_df.rename(columns = {'Abbrev': 'state_abbrev'})
states_abbrev_df.head()

,Airport,City,state_abbrev,State,IATA
0,Aberdeen Regional Airport,Aberdeen,SD,South Dakota,ABR
1,Abilene Regional Airport,Abilene,TX,Texas,ABI
2,Abraham Lincoln Capital Airport,Springfield,IL,Illinois,SPI
3,Akron-Canton Regional Airport,Akron / Canton,OH,Ohio,CAK
4,Albany International Airport,Albany,NY,New York,ALB


In [4]:
# API Call for covid data 
covid_base_url = "https://api.covidtracking.com/v1/states/daily.json"
response = requests.get(covid_base_url).json()

#print(json.dumps(response, indent=4))

# pull the data from a source df to a target df by creating a new df

covid_df = pd.DataFrame(response, columns=['date', 'state', 'positive', 'death', 'hospitalized', 'totalTestResults'])
covid_df.head()



,date,state,positive,death,hospitalized,totalTestResults
0,20201021,AK,12644.0,68.0,NaN,547140.0
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0
3,20201021,AS,0.0,0.0,NaN,1616.0
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0


In [5]:
covid_df = covid_df.rename(columns = {'state': 'state_abbrev'})
covid_df.head()

,date,state_abbrev,positive,death,hospitalized,totalTestResults
0,20201021,AK,12644.0,68.0,NaN,547140.0
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0
3,20201021,AS,0.0,0.0,NaN,1616.0
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0


In [6]:
covid_df['Date'] = ''
covid_df.head()

        

,date,state_abbrev,positive,death,hospitalized,totalTestResults,Date
0,20201021,AK,12644.0,68.0,NaN,547140.0,
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0,
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0,
3,20201021,AS,0.0,0.0,NaN,1616.0,
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0,


In [7]:

for x, y in covid_df.iterrows():
    
    org_date = str(y.get(key='date'))
    
    year_s = org_date[0:4]
    month_s = org_date[4:6]
    day_s = org_date[6:8]
    formatted_date = f"{year_s}-{month_s}-{day_s}"
    
    covid_df.at[x, 'Date'] = str(formatted_date)
  
    
        
covid_df = covid_df.drop('date', axis=1)
covid_df.head(20)


,state_abbrev,positive,death,hospitalized,totalTestResults,Date
0,AK,12644.0,68.0,NaN,547140.0,2020-10-21
1,AL,174528.0,2805.0,19330.0,1273523.0,2020-10-21
2,AR,101596.0,1751.0,6526.0,1242752.0,2020-10-21
3,AS,0.0,0.0,NaN,1616.0,2020-10-21
4,AZ,233912.0,5854.0,20820.0,1657389.0,2020-10-21
5,CA,877784.0,17027.0,NaN,17293139.0,2020-10-21
6,CO,87582.0,2060.0,8332.0,1724038.0,2020-10-21
7,CT,64871.0,4567.0,12043.0,2070126.0,2020-10-21
8,DC,16498.0,642.0,NaN,476443.0,2020-10-21
9,DE,23375.0,670.0,NaN,331480.0,2020-10-21


In [8]:
# API Call for flights data 

url = f"http://api.aviationstack.com/v1/flights?access_key={api_key}&"
query_date = ['2020-10-01','2020-10-02','2020-10-03' ]
query_iata = ['JFK', 'SAN']
value_list = []

for q_date in query_date:
    for q_iata in query_iata:
        flights_base_url = f"{url}flight_date={q_date}&arr_iata={q_iata}&flight_status=landed&limit=1"
        flights_response = requests.get(flights_base_url).json()
        landed_flights = flights_response['pagination']['total']
        city = flights_response['data'][0]['arrival']['iata']
        #city = city.split('/')[1]
        #city = city.replace('_', ' ')
       
        value_list.append({'Date': q_date, 'IATA':q_iata, 'Total Landed Flights': landed_flights})
        
flights_df = pd.DataFrame(value_list)   
flights_df


,Date,IATA,Total Landed Flights
0,2020-10-01,JFK,840
1,2020-10-01,SAN,210
2,2020-10-02,JFK,821
3,2020-10-02,SAN,206
4,2020-10-03,JFK,744
5,2020-10-03,SAN,192


In [9]:
#df1.merge(df2,on='name').merge(df3,on='name')
merged_df = states_abbrev_df.merge(flights_df, on='IATA')
merged_df.head(50)

,Airport,City,state_abbrev,State,IATA,Date,Total Landed Flights
0,John F. Kennedy International Airport,New York,NY,New York,JFK,2020-10-01,840
1,John F. Kennedy International Airport,New York,NY,New York,JFK,2020-10-02,821
2,John F. Kennedy International Airport,New York,NY,New York,JFK,2020-10-03,744
3,San Diego International Airport,San Diego,CA,California,SAN,2020-10-01,210
4,San Diego International Airport,San Diego,CA,California,SAN,2020-10-02,206
5,San Diego International Airport,San Diego,CA,California,SAN,2020-10-03,192


In [10]:
merged_df = merged_df.merge(covid_df, how='inner', left_on=["Date", "state_abbrev"], right_on=["Date","state_abbrev"])
merged_df

,Airport,City,state_abbrev,State,IATA,Date,Total Landed Flights,positive,death,hospitalized,totalTestResults
0,John F. Kennedy International Airport,New York,NY,New York,JFK,2020-10-01,840,460031.0,25490.0,89995.0,10856531.0
1,John F. Kennedy International Airport,New York,NY,New York,JFK,2020-10-02,821,461629.0,25497.0,89995.0,10976024.0
2,John F. Kennedy International Airport,New York,NY,New York,JFK,2020-10-03,744,463360.0,25505.0,89995.0,11110291.0
3,San Diego International Airport,San Diego,CA,California,SAN,2020-10-01,210,813687.0,15888.0,NaN,14771851.0
4,San Diego International Airport,San Diego,CA,California,SAN,2020-10-02,206,817277.0,15986.0,NaN,14868431.0
5,San Diego International Airport,San Diego,CA,California,SAN,2020-10-03,192,819436.0,16074.0,NaN,14998888.0
